In [5]:
import tensorflow as tf
from tensorflow.keras import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [19]:
(X,Y), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data()

In [22]:
x_train, x_val, y_train, y_val = train_test_split(X,Y, test_size= 0.2)
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()

sc.fit(x_train)
x_train = sc.transform(x_train)
x_test = sc.transform(x_test)
x_val = sc.transform(x_val)

## this is the check if the values are too big to be used. if they are you normalize it
x_train.max()
x_train.min()


9.313811273237395

In [23]:
x_train.shape

## construct the 
## for regression, we do not use to activation = "softmax"
## for regression, we do not use Conv2D because it is not an image

input_to_model = tf.keras.layers.Input(shape = (13,), name = "Input_to_model")
x = tf.keras.layers.Dense(256, activation = "relu")(input_to_model)
x = tf.keras.layers.Dense(512, activation= "relu")(x)

output = tf.keras.layers.Dense(10, name = "output_layer")(x)


In [24]:
output.shape

## train model
model = tf.keras.Model(inputs = input_to_model, outputs = output)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_to_model (InputLayer)  [(None, 13)]             0         
                                                                 
 dense_2 (Dense)             (None, 256)               3584      
                                                                 
 dense_3 (Dense)             (None, 512)               131584    
                                                                 
 output_layer (Dense)        (None, 10)                5130      
                                                                 
Total params: 140,298
Trainable params: 140,298
Non-trainable params: 0
_________________________________________________________________


In [25]:
## after training, we have to account for optimizer(Adam), loss(mean_square error(the lower the better))   
## r2 square error is used after training the data

model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=1e-3), loss = "mse")

In [29]:
## we train the data and test the validation

model_history = model.fit(x = x_train, y = y_train, validation_data = (x_val, y_val), batch_size= 32, epochs=100)

Epoch 1/100
11/11 [==============================] - 0s 12ms/step - loss: 2.7424 - val_loss: 8.7948
Epoch 2/100
11/11 [==============================] - 0s 8ms/step - loss: 2.1103 - val_loss: 8.5888
Epoch 3/100
11/11 [==============================] - 0s 8ms/step - loss: 2.0966 - val_loss: 7.9612
Epoch 4/100
11/11 [==============================] - 0s 10ms/step - loss: 2.1565 - val_loss: 8.4310
Epoch 5/100
11/11 [==============================] - 0s 8ms/step - loss: 2.2135 - val_loss: 8.6173
Epoch 6/100
11/11 [==============================] - 0s 9ms/step - loss: 2.6880 - val_loss: 7.5691
Epoch 7/100
11/11 [==============================] - 0s 8ms/step - loss: 2.5682 - val_loss: 8.4028
Epoch 8/100
11/11 [==============================] - 0s 9ms/step - loss: 2.0060 - val_loss: 7.8400
Epoch 9/100
11/11 [==============================] - 0s 8ms/step - loss: 1.9666 - val_loss: 8.4135
Epoch 10/100
11/11 [==============================] - 0s 8ms/step - loss: 1.8986 - val_loss: 8.0803
Epoch 1

In [30]:
## model evalution
test_score = model.evaluate(x_test, y_test)

4/4 [==============================] - 0s 3ms/step - loss: 5312.7876
